# Capstone, Week 3 Assignment, Part 2 (Also includes part 1)

#### Import libraries

In [1]:
import pandas as pd
import numpy as np

#### Load data from wikipedia

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

In [3]:
df.head(3)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


#### Check for missing data

In [4]:
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

Postal Code
False    180
Name: Postal Code, dtype: int64

Borough
False    180
Name: Borough, dtype: int64

Neighborhood
False    180
Name: Neighborhood, dtype: int64



#### Drop rows that do not have an assigned borough

In [5]:
df = df[df.Borough != 'Not assigned']

In [6]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Verify there are no repeated Postal codes

In [7]:
print('There are',df['Postal Code'].count() - df['Postal Code'].nunique(),'repeated postal codes')

There are 0 repeated postal codes


#### Replace unassigned neighborhoods with their Borough name

In [8]:
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])

In [9]:
df.head(5)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Get the size of the cleansed data set

In [10]:
df.shape

(103, 3)

## Part 2 - Getting geospatial data

#### Install geocoder

#### Get coordinates

#### Geocoder not responding, so load csv data

In [11]:
filename = "http://cocl.us/Geospatial_data"
df_coords = pd.read_csv(filename)



#### Combine Lat/Long data with original dataframe

In [12]:
df_merge = pd.merge(df,
                 df_coords[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code',
                 how='left')



In [13]:
df_merge.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Begin K-means clustering

#### Load libraries

In [18]:
import random 
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

#### Scatter plot of Latitude/Longitude Data

In [22]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011


#### Create cluster (using 4 clusters), and then merge clusters with original dataset

In [63]:
#Define clustering
k_means = KMeans(init = "k-means++", n_clusters = 4, n_init = 12)

#Create arrays for lat and long
X = df_merge[['Latitude','Longitude']].to_numpy()
k_means.fit(X)


k_means_labels = k_means.labels_

#Merge cluster data with original dataset (only run once)
#df_cluster = df_merge.insert(0, 'Cluster Labels', k_means.labels_)

k_means_cluster_centers = k_means.cluster_centers_

df_merge.head()

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,1,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Neighborhood'], df_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters